In [ ]:
import os
import nibabel as nib
import numpy as np
from scipy.ndimage import label, center_of_mass
import matplotlib.pyplot as plt



In [ ]:
# Path to the binary mask NIfTI file
dir = "/usagers4/u139017/Proyecto/Connected components?/tibia_111801.nii.gz"
# loads
nii = nib.load(dir)
data = nii.get_fdata()

# labels connected components
labeled, num = label(data)

print(f"found {num} connected components")

# Calcular centroides 
centroids = center_of_mass(data, labeled, range(1, num+1))

# sort by x coordinate
order = sorted(range(num), key=lambda i: centroids[i][0])  # X from left to right

# create masks
left_mask = (labeled == (order[0] + 1)).astype(np.uint8)
right_mask = (labeled == (order[1] + 1)).astype(np.uint8)

# savve masks
left_img = nib.Nifti1Image(left_mask, nii.affine)
right_img = nib.Nifti1Image(right_mask, nii.affine)
nib.save(left_img, "tibia_ejemplo_izquierda.nii.gz")
nib.save(right_img, "tibia_ejemplo_derecha.nii.gz")


Se encontraron 2 componentes conectados


ADD pre visualization for every subject

In [ ]:

input_folder = "/usagers4/u139017/Proyecto/Connected components?/prueba_tibia"       
output_folder = os.path.join(input_folder, "separated")  # subfolder for results
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.startswith("tibia_") and filename.endswith(".nii.gz"):
        print(f"Procesando {filename}")
        path = os.path.join(input_folder, filename)
        nii = nib.load(path)
        data = nii.get_fdata()

        # Connected components
        labeled, num = label(data)
        if num != 2:
            print(f"  Se encontraron {num} componentes en {filename}, se esperaba 2. Saltando...")
            continue

        # Centroids calculation
        centroids = center_of_mass(data, labeled, range(1, num+1))
        # sort by X coordinate
        order = sorted(range(num), key=lambda i: centroids[i][0])

        # Since X axis goes from left to right
        idx_left = order[1] + 1  # Componente con mayor X
        idx_right = order[0] + 1 # Componente con menor X

        # Crear MASKS
        left_mask = (labeled == idx_left).astype(np.uint8)
        right_mask = (labeled == idx_right).astype(np.uint8)

        # saves
        base = filename.replace("tibia_", "").replace(".nii.gz", "")
        nib.save(nib.Nifti1Image(left_mask, nii.affine), os.path.join(output_folder, f"left_tibia_{base}.nii.gz"))
        nib.save(nib.Nifti1Image(right_mask, nii.affine), os.path.join(output_folder, f"right_tibia_{base}.nii.gz"))

        # Slice medio del eje Z
        z_mid = data.shape[2] // 2

        # Obtener slices 2D
        orig_slice = data[:, :, z_mid]
        left_slice = left_mask[:, :, z_mid]
        right_slice = right_mask[:, :, z_mid]

        # Crear imagen RGB
        overlay = np.zeros(orig_slice.shape + (3,), dtype=np.uint8)
        overlay[..., 0] = (left_slice > 0) * 255   # Red for left tibia
        overlay[..., 1] = (right_slice > 0) * 255  # Green for right tibia

        # Plot y guardar
        plt.figure(figsize=(5, 5))
        plt.imshow(orig_slice, cmap='gray')        # Or cmap='bone' 
        plt.imshow(overlay, alpha=0.5)
        plt.axis('off')
        plt.title(f"{base} - corte Z medio")
        plt.tight_layout()

        plt.savefig(os.path.join(output_folder, f"preview_{base}.png"), dpi=150)
        plt.close()

print("✅ finished.")


Procesando tibia_111539.nii.gz
Procesando tibia_111801.nii.gz
Procesando tibia_112076.nii.gz
✅ Procesamiento por lote terminado.
